# Data wrangling with Pandas

## File I/O

### Reading text files

Pandas provides a bunch of utilities to read and write to text formats rivaling those provided by R. The workhorse functions are `read_csv` and `read_table` analogous to those by the same name in R. Other available options are `read_fwf` and `read_clipboard`, etc. On the other hand, `pandas` also provides functions to read formats such as `html`, `json`, sql tables etc.

In this session, we briefly discuss the `read_table` and `read_excel` functions to load data first.

#### `read_table`

In [1]:
import pandas as pd
import numpy as np

?pd.read_table

In [2]:
# Let's go back to our data on Pythagorean triplets:

pyTrips = pd.read_table("../../../data/pythagorean-triples.txt",
                        sep=",", names=["base", "altitude", "hypotenuse"])
pyTrips.head()

,base,altitude,hypotenuse
0,3,4,5
1,5,12,13
2,15,8,17
3,7,24,26
4,21,20,29


In [3]:
pyTrips['IsPythagorean'] = (pyTrips['base'] ** 2) + \
                           (pyTrips['altitude'] ** 2) == \
                           (pyTrips['hypotenuse'] ** 2)
pyTrips.head()

,base,altitude,hypotenuse,IsPythagorean
0,3,4,5,True
1,5,12,13,True
2,15,8,17,True
3,7,24,26,False
4,21,20,29,True


In [4]:
?pd.DataFrame.to_csv

In [5]:
pyTrips.to_csv(path_or_buf="../../../data/is-pythagorean.tsv", sep="\t")

In [6]:
!head -10 ../../../data/is-pythagorean.tsv

	base	altitude	hypotenuse	IsPythagorean
0	3	4	5	True
1	5	12	13	True
2	15	8	17	True
3	7	24	26	False
4	21	20	29	True
5	35	12	37	True
6	9	40	41	True
7	45	28	52	False
8	11	60	61	True


This section just introduces reading and writing text files using Pandas. There is **much** more detail on this available in the McKinney book that you should refer to for details and recipes.

#### `read_excel`

Python provides the capability to read Excel files using external python modules called `xlrd`, `openpyxl`. One will need to install the module first to use this functionality. Let's look at an example.

In [7]:
?pd.read_excel

In [8]:
trips = pd.read_excel("../../../data/pythagorean-triples.xlsx", sheetname="triples")
trips.head()

,base,altitude,hypotenuse
0,3,4,5
1,5,12,13
2,15,8,17
3,7,24,26
4,21,20,29


In [9]:
ans1 = pd.read_excel("../../../data/pythagorean-triples.xlsx", sheetname="ans1")
ans2 = pd.read_excel("../../../data/pythagorean-triples.xlsx", sheetname="ans2")

print(ans1.head())
print(ans2.head())

  IsPythagorean
0          True
1          True
2          True
3         False
4          True
  IsPythagorean
0          True
1          True
2          True
3         False
4          True


**Exercise**: Add a column to ans2 that takes the value 1 if the corresponding row in ans1 is correct and 0 otherwise. Then find the total score for ans2. Do the same for ans1. Who got a higher score?

In [10]:
results = pd.DataFrame({"Ans1": 101, "Ans2": 96}, index=[0])
results

,Ans1,Ans2
0,101,96


In [11]:
#writer = pd.ExcelWriter('../../../data/scores.xlsx')
#results.to_excel(writer)

## Merging DataFrames

Merging or joining databases is to combine information from two different sources on a common index. There are multiple versions of joins: _left_ , _right_, _inner_, and _outer_.

Pandas provides a `merge` function that allows each of these type of indexing to occur. Let's create two `DataFrames` to look at examples.

In [12]:
from string import ascii_lowercase, ascii_uppercase
import random

letters = list(ascii_lowercase)
LETTERS = list(ascii_uppercase)
random.seed(3.141)

In [13]:
df1 = pd.DataFrame({random.choice(letters): {
                      'x': random.uniform(0, 1),
                      'y': random.choice(LETTERS)
                    } for x in range(20)}).T
df1

,x,y
c,0.1365529,V
d,0.2421965,U
e,0.3272484,S
g,0.3252303,B
i,0.6627221,X
j,0.8019757,A
m,0.1864096,R
n,0.492297,E
o,0.8691006,W
p,0.1106273,C


In [14]:
df2 = pd.DataFrame({random.choice(letters): {
                      'a': random.uniform(0, 1),
                      'b': random.choice(LETTERS)
                    } for x in range(20)}).T
df2

,a,b
a,0.5824532,G
b,0.4106862,O
e,0.7102085,N
h,0.4833733,M
i,0.5855433,T
j,0.8316282,D
r,0.3818659,E
t,0.8192802,Z
u,0.7930255,C
v,0.3119217,A


In [15]:
pd.merge(df1, df2, how='left',
         left_index=True, right_index=True)

,x,y,a,b
c,0.1365529,V,NaN,NaN
d,0.2421965,U,NaN,NaN
e,0.3272484,S,0.7102085,N
g,0.3252303,B,NaN,NaN
i,0.6627221,X,0.5855433,T
j,0.8019757,A,0.8316282,D
m,0.1864096,R,NaN,NaN
n,0.492297,E,NaN,NaN
o,0.8691006,W,NaN,NaN
p,0.1106273,C,NaN,NaN


In [16]:
pd.merge(df1, df2, how='right',
         left_index=True, right_index=True)

,x,y,a,b
a,NaN,NaN,0.5824532,G
b,NaN,NaN,0.4106862,O
e,0.3272484,S,0.7102085,N
h,NaN,NaN,0.4833733,M
i,0.6627221,X,0.5855433,T
j,0.8019757,A,0.8316282,D
r,0.8170989,B,0.3818659,E
t,0.310661,M,0.8192802,Z
u,0.6443058,H,0.7930255,C
v,NaN,NaN,0.3119217,A


In [17]:
pd.merge(df1, df2, how='inner',
         left_index=True, right_index=True)

,x,y,a,b
e,0.3272484,S,0.7102085,N
i,0.6627221,X,0.5855433,T
j,0.8019757,A,0.8316282,D
r,0.8170989,B,0.3818659,E
t,0.310661,M,0.8192802,Z
u,0.6443058,H,0.7930255,C
z,0.04010426,T,0.342259,E


In [18]:
pd.merge(df1, df2, how='outer',
         left_index=True, right_index=True)

,x,y,a,b
a,NaN,NaN,0.5824532,G
b,NaN,NaN,0.4106862,O
c,0.1365529,V,NaN,NaN
d,0.2421965,U,NaN,NaN
e,0.3272484,S,0.7102085,N
g,0.3252303,B,NaN,NaN
h,NaN,NaN,0.4833733,M
i,0.6627221,X,0.5855433,T
j,0.8019757,A,0.8316282,D
m,0.1864096,R,NaN,NaN


In [19]:
pd.merge(df1, df2, how='inner',
         left_on='y', right_on='b')

,x,y,a,b
0,0.2421965,U,0.2409431,U
1,0.8019757,A,0.3119217,A
2,0.492297,E,0.3818659,E
3,0.492297,E,0.342259,E
4,0.1106273,C,0.7930255,C
5,0.310661,M,0.4833733,M
6,0.04010426,T,0.5855433,T


**Exercise**: In df1, add a column `z` to contain the uppercase version of the index of df1. Now, find the _inner_ join of df1 and df2 on columns `z` and `b`. 

## Concatenating DataFrames

Concatenation, stacking, or _binding_ (a la `rbind`, `cbind` in R) is the transformation of combining two datasets together. In Pandas, the `pandas.concat` function provides this capability. Let's look at a few examples.

In [20]:
pd.concat((df1, df2))

,a,b,x,y
c,NaN,NaN,0.1365529,V
d,NaN,NaN,0.2421965,U
e,NaN,NaN,0.3272484,S
g,NaN,NaN,0.3252303,B
i,NaN,NaN,0.6627221,X
j,NaN,NaN,0.8019757,A
m,NaN,NaN,0.1864096,R
n,NaN,NaN,0.492297,E
o,NaN,NaN,0.8691006,W
p,NaN,NaN,0.1106273,C


In [21]:
pd.concat((df1, df2), axis=1) # Equivalent to outer join on the indices.

,x,y,a,b
a,NaN,NaN,0.5824532,G
b,NaN,NaN,0.4106862,O
c,0.1365529,V,NaN,NaN
d,0.2421965,U,NaN,NaN
e,0.3272484,S,0.7102085,N
g,0.3252303,B,NaN,NaN
h,NaN,NaN,0.4833733,M
i,0.6627221,X,0.5855433,T
j,0.8019757,A,0.8316282,D
m,0.1864096,R,NaN,NaN


In [22]:
pd.concat((df1, df2), axis=1, join="inner")

,x,y,a,b
e,0.3272484,S,0.7102085,N
i,0.6627221,X,0.5855433,T
j,0.8019757,A,0.8316282,D
r,0.8170989,B,0.3818659,E
t,0.310661,M,0.8192802,Z
u,0.6443058,H,0.7930255,C
z,0.04010426,T,0.342259,E


## Reshaping / Pivoting

Reshaping or pivoting involves essentially two primary operations: a) `stack`: reshaping data from column (wide) to rows (long) and b) `unstack`: reshaping data from rows (long) to columns (wide).

In [23]:
wide = pd.DataFrame(np.arange(6).reshape((2, 3)),
                    index=pd.Index(['Ohio', 'Colorado'], name='state'),
                    columns=pd.Index(['one', 'two', 'three'], name='number'))
wide

number,one,two,three
state,,,
Ohio,0,1,2
Colorado,3,4,5


In [24]:
long = wide.stack() # Takes the data from wide to long using hierarchical indexing.
long

state     number
Ohio      one       0
          two       1
          three     2
Colorado  one       3
          two       4
          three     5
dtype: int64

In [25]:
long.unstack() # Goes from long to wide

number,one,two,three
state,,,
Ohio,0,1,2
Colorado,3,4,5


By defualt, unstack would pivot along the innermost-level of the hierarchy: in this case number. However, we may over-ride that behavior by specifying an index (either by level or by name).

In [26]:
print(long.unstack('state'), "\n\n", long.unstack(0), "\n\n", long.unstack('number'))

state   Ohio  Colorado
number                
one        0         3
two        1         4
three      2         5 

 state   Ohio  Colorado
number                
one        0         3
two        1         4
three      2         5 

 number    one  two  three
state                    
Ohio        0    1      2
Colorado    3    4      5


Going from `long` to `wide` format can also be achieved in a simple manner using the `pivot` function. Let's look at a sample long-dataset.

In [27]:
pd.DataFrame({'roll#': 1,
              'triple#': range(101),
              'answer': np.random.permutation(ans1.values).reshape((101,))
             }, columns=['roll#', 'triple#', 'answer'])

,roll#,triple#,answer
0,1,0,False
1,1,1,True
2,1,2,True
3,1,3,True
4,1,4,True
5,1,5,False
6,1,6,True
7,1,7,True
8,1,8,True
9,1,9,True


In [28]:
answers = [pd.DataFrame({'roll#': x + 1,
                         'triple#': range(101),
                         'answer': np.random.permutation(ans1.values).reshape((101,))
                        }, columns=['roll#', 'triple#', 'answer']) for 
           x in range(10)]
answers = pd.concat(answers)
print(answers.head(), "\n\n", answers.tail())

   roll#  triple# answer
0      1        0   True
1      1        1   True
2      1        2   True
3      1        3   True
4      1        4   True 

      roll#  triple# answer
96      10       96   True
97      10       97   True
98      10       98  False
99      10       99  False
100     10      100   True


What we would like to achieve with this dataset is to reshape it to wide where we have the roll number along the rows, and each triple as a column of it's own; the value being the answer. Pivoting like this is easily achieved using `DataFrame.pivot`.

In [29]:
wideAnswers = answers.pivot('roll#', 'triple#', 'answer')
wideAnswers

triple#,0,1,2,3,4,5,6,7,8,9,...,91,92,93,94,95,96,97,98,99,100
roll#,,,,,,,,,,,,,,,,,,,,,
1,True,True,True,True,True,True,True,True,True,True,...,True,True,True,False,False,True,True,True,True,True
2,True,True,False,True,True,True,True,True,True,True,...,True,True,True,True,True,True,True,True,True,False
3,True,True,True,False,True,True,False,True,True,False,...,True,True,True,True,True,True,True,True,True,True
4,True,False,True,True,True,True,True,True,True,True,...,True,True,True,True,True,True,True,True,True,True
5,True,True,False,True,True,True,True,True,True,True,...,True,True,True,False,False,True,False,True,True,True
6,True,True,True,False,True,False,True,True,True,True,...,True,True,True,False,True,True,True,True,False,True
7,True,True,True,True,True,True,True,True,True,False,...,False,True,True,True,True,True,True,True,True,True
8,True,True,True,True,True,True,True,True,True,True,...,True,True,True,False,False,True,True,True,False,True
9,False,True,True,True,True,True,True,True,True,True,...,True,True,True,True,True,True,True,True,False,True


In [30]:
wideAnswers.stack() # Going back to long.

roll#  triple#
1      0           True
       1           True
       2           True
       3           True
       4           True
       5           True
       6           True
       7           True
       8           True
       9           True
       10          True
       11          True
       12          True
       13          True
       14          True
       15          True
       16          True
       17          True
       18          True
       19          True
       20          True
       21          True
       22          True
       23          True
       24         False
       25          True
       26          True
       27          True
       28          True
       29          True
                  ...  
10     71          True
       72         False
       73          True
       74          True
       75          True
       76          True
       77          True
       78          True
       79          True
       80          True
 

## Data transformations

### Removing duplicates

Pandas has two useful functions called `duplicated` and `drop_duplicates` that are useful to detect and remove duplicates from a dateset or its columns. By default, these functions match on every column in the dataset to detect duplicates. Let's take the example of our `answers` for the Pythagorean triples and check if there are any duplicates in the data.

In [31]:
answers.duplicated().head()

0    False
1    False
2    False
3    False
4    False
dtype: bool

In [32]:
answers.duplicated().any() # No duplicates.

False

However, if we consider duplicates only on columns `roll#` and `answer`, we will likely find many.

In [33]:
answers.duplicated(['roll#', 'answer']).head()

0    False
1     True
2     True
3     True
4     True
dtype: bool

In [34]:
answers.drop_duplicates(['roll#', 'answer'])

,roll#,triple#,answer
0,1,0,True
24,1,24,False
0,2,0,True
2,2,2,False
0,3,0,True
3,3,3,False
0,4,0,True
1,4,1,False
0,5,0,True
2,5,2,False


The determination of duplicates can work by scanning the data from the beginning or the end. By default, the functions above work from the beginning but can be provided an optional argument to start looking from the end. For example:

In [35]:
answers.drop_duplicates(['roll#', 'answer'], take_last=True)

,roll#,triple#,answer
95,1,95,False
100,1,100,True
99,2,99,True
100,2,100,False
77,3,77,False
100,3,100,True
90,4,90,False
100,4,100,True
97,5,97,False
100,5,100,True


### Mappings

Mappings are transformations that _map_ a value in the input data to an output value using a set of associations (`dict`) or a set of rules (`function`). Columns in a Pandas DataFrame can be easily mapped using a function or a dictionary using the `.map()` method.

Suppose, for the set of roll numbers, we also had a set of names that we would like to add to the `answers` DataFrame.

In [36]:
rolls2names = {
    1: 'alpha', 2: 'bravo', 3: 'charlie', 4: 'delta', 5: 'jane',
    6: 'john', 7: 'johnny', 8: 'ed', 9: 'edd', 10: 'eddy'
}
answers['name'] = answers['roll#'].map(rolls2names)
answers.tail()

,roll#,triple#,answer,name
96,10,96,True,eddy
97,10,97,True,eddy
98,10,98,False,eddy
99,10,99,False,eddy
100,10,100,True,eddy


**Exercise**: Pivot the `answers` table such that it has `names` as the columns and `triple#` as the rows and the `answer` as values.

#### Replacing values

Replacing values can be thought of as a special case of a `map` transformation where instead of mapping each input to an output, we only map some and the others are left unchnaged. For example, let's consider that the role number for a certain student has changed, then one may replace the earlier roll number with the new one in our data while not affecting the others. This may even be done for multiple values using a dictionary.

In [37]:
answers['roll#'] = answers['roll#'].replace(10, 12)
answers.tail()

,roll#,triple#,answer,name
96,12,96,True,eddy
97,12,97,True,eddy
98,12,98,False,eddy
99,12,99,False,eddy
100,12,100,True,eddy


In [38]:
answers['name'] = answers['name'].replace({'johnny': 'johnny bravo', 'jane': 'jane doe'})
answers[answers['roll#'].isin((5, 7))]

,roll#,triple#,answer,name
0,5,0,True,jane doe
1,5,1,True,jane doe
2,5,2,False,jane doe
3,5,3,True,jane doe
4,5,4,True,jane doe
5,5,5,True,jane doe
6,5,6,True,jane doe
7,5,7,True,jane doe
8,5,8,True,jane doe
9,5,9,True,jane doe


### Indicator / dummy variables


A common problem in data analysis is to split a categorical variable into a bunch of dummy variables. Pandas has a function called `get_dummies` which does exactly this in a simple way.

In [39]:
# Let's look at what df1 looked like
df1

,x,y
c,0.1365529,V
d,0.2421965,U
e,0.3272484,S
g,0.3252303,B
i,0.6627221,X
j,0.8019757,A
m,0.1864096,R
n,0.492297,E
o,0.8691006,W
p,0.1106273,C


In [40]:
pd.get_dummies(df1['y']) # Get dummy variables for the column 'y'

,A,B,C,E,H,M,R,S,T,U,V,W,X,Y
c,0,0,0,0,0,0,0,0,0,0,1,0,0,0
d,0,0,0,0,0,0,0,0,0,1,0,0,0,0
e,0,0,0,0,0,0,0,1,0,0,0,0,0,0
g,0,1,0,0,0,0,0,0,0,0,0,0,0,0
i,0,0,0,0,0,0,0,0,0,0,0,0,1,0
j,1,0,0,0,0,0,0,0,0,0,0,0,0,0
m,0,0,0,0,0,0,1,0,0,0,0,0,0,0
n,0,0,0,1,0,0,0,0,0,0,0,0,0,0
o,0,0,0,0,0,0,0,0,0,0,0,1,0,0
p,0,0,1,0,0,0,0,0,0,0,0,0,0,0


**Exercise**: A common transformation of variables is to take a numeric variable and convert it to a set of dummy variables indicating the interval in which the numeric variable falls for a given record. Let's create the following example:

In [41]:
heights = pd.DataFrame({'name': list(rolls2names.values()),
                        'height': np.random.uniform(150, 210, 10)},
                       columns=['name', 'height'])
heights

,name,height
0,alpha,151.417568
1,bravo,192.934822
2,charlie,188.379681
3,delta,158.864330
4,jane,209.743464
5,john,196.470798
6,johnny,183.375794
7,ed,152.293439
8,edd,164.981267
9,eddy,168.713312


Look at the documentation for `?pd.cut` and add a variable called `category` to the DataFrame `heights` such that: category takes the value `short` if height < 170, `average` if 170 <= height < 190, `tall` otherwise. Then create indicator variables for the `category` variable.